# Introduction à la librairie Arcade

[Arcade](https://arcade.academy/index.html) est une librairie Python pour la création de jeux. 

C'est une librairie «haut niveau» (plus facile) basée sur la librairie «bas niveau» [pyglet](https://pyglet.readthedocs.io/en/latest/). Elle est conçue pour **faciliter la programmation de jeu** en Python.

Pour l'installer sous windows (je suppose que Python est déjà installé), saisir dans `cmd.exe` (console windows)

    python -m pip install arcade

Sa documentation contient une foule d'exemples et quelques tutoriels pour démarrer. Elle contient aussi la documentation de toutes les classes, fonctions (ainsi que leur code source) qu'elle propose.

Ici, je reprends le [tutoriel de la documentation appelé «Simple Plateformer»](https://arcade.academy/examples/platform_tutorial/index.html#) (simple jeu de plateforme).

Pour vous faciliter la vie, je traduis une partie du code mais je vous recommande vivement de comparer avec la version d'origine en anglais pour vous familiariser avec le vocabulaire courant du monde des jeux...

##  Une fenêtre de jeu - *1_main.py*

La structure de base du code arcade pour produire une fenêtre est:

```python
import arcade

# Constantes
LARGEUR_ECRAN = 1000
#...


class MonJeu(arcade.Window):
    """
    Classe principale de l'application: représente une fenêtre de jeu customisée.
    qui dérive de la fenêtre de base d'arcade `arcade.Window`.
    """

    def __init__(self):
        # Initialise la fenêtre en appelant la méthode __init__ de la classe mère
        # `arcade.Window` (d'où l'utilisation de `super`())
        super().__init__(LARGEUR_ECRAN,...)

        # Choisir la couleur de fond
        arcade.set_background_color(...)

    def setup(self):
        """ Configurer le jeu ici. Appeler cette fonction pour (re)démarrer le jeu."""
        pass

    def on_draw(self):
        """ Affichage à l'écran """

        arcade.start_render()
        # Le code pour dessiner à l'écran:


def main():
    """ fonction principale pour lancer le jeu """
    window = MonJeu()  # création de la fenêtre ...
    window.setup()  # ... puis configuration ...
    arcade.run()  # lance la boucle de gestion des événements (la «mainloop» ...)


if __name__ == "__main__":
    main()
```

On distingue quatre zones:
1. Import d'arcade et déclaration de constantes (par convention en majuscule),


2. Définition de la classe `MonJeu`:
    - elle représente une fenêtre Arcade (`arcade.Window`) spécialisée pour notre jeu,
    - c'est à l'intérieur de cette classe qu'on va coder effectivement le jeu,


3. Définition d'une fonction `main` («principale») dont le rôle est simplement de:
    - créer notre fenêtre de Jeu,
    - de le configurer (via son opération `setup`)
    - et de lancer la boucle principale d'arcade (s'occupe de gérer les événements utilisateurs, rafraîchir l'écran etc.)


4. `if __name__ ...`: sert simplement a **appeler** la fonction `main` pour démarrer le jeu.

#### Note complémentaire sur `if __name__` ...

Lorsqu'on lance **directement** un fichier python *test.py*, la variable *interne* `__name__` vaut `"__main__"`.

En revanche, lorsqu'on **importe** un fichier python - `import test` (inutile de mettre le *.py*) depuis un autre fichier python `autre.py` et qu'on lance ce dernier, alors quand python lit `test.py`, la variable __name__ ne vaut pas "__main__".

Cela permet de distinguer les fichiers lancés *directement* ou *indirectement* (via un import). Voilà une expérience à faire:

Dans *test.py*
```python
print(f"test.py: __name__ vaut {__name__}")
if __name__ == "__main__":
    print("Salut toi!")
```

Dans *autre.py*
```python
import test # doit-être dans le même répertoire que moi!
print(f"autre.py: __name__ vaut {__name__}")
```

Ensuite, on lance python sur chaque fichier (ligne de commande):

    python test.py 
        test.py: __name__ vaut __main__ 
        Salut toi!
    
    python autre.py 
        test.py: __name__ vaut test
        autre.py: __name__ vaut __main__

### `__init__(self, ...)` ???

Il s'agit d'une fonction spéciale de Python qui est appelée automatiquement lors de la construction de la fenêtre de jeu `window = MonJeu()`.

Elle sert à initialiser les attributs propres à cette fenêtre.

### `self` ???

Sans entrer dans les détails, `self` représente la fenêtre de jeu *courante* (tandis que la classe `MonJeu` sert à créer cette fenêtre).

On peut utiliser `self` pour lui ajouter des **attributs** (variables attachées à la fenêtre): 
- déclaration `self.mon_attribut = valeur`,
- accès à l'attribut `self.mon_attribut`.

## Planter le décors - *2_main.py*

On utilise des images qu'Arcade nous fournit par défaut (voir...). Elles vont nous servir à:
1. Créer des **Sprites** individuels, ce sont des images destinées à être animées:
    - `mon_sprite = arcade.Sprite(chemin_img, echelle, ...)` puis
    - on le positionne sur l'écran: `mon_sprite.center_x = x` (et pareil pour y)


2. Grouper ces Sprites dans des listes adaptée `SpriteList`: 
    - `grp_sprites = arcade.SpriteList(options)` puis
    - `grp_sprites.append(<un sprite>)`,


3. Dessiner ces sprites à l'écran: `grp_sprites.draw()` (dans `on_draw`)

Voici des extraits de code pertinent pour le décors (formé de «tuiles» \[*tiles*\]):
```python
    def __init__(self):
        ...
        self.tuiles = None # déclaration de l'attribut «tuiles» pour la fenêtre courante
        ...
    
    def setup(self):
        ...
        self.tuiles = arcade.SpriteList()
        ...
        for x in ...:
            bloc_pelouse = arcade.Sprite(
                ":resources:images/tiles/grassMid.png", # chemin
                ECHELLE_TUILE # constante d'échelle
            )
            # positionnement
            bloc_pelouse.center_x = x
            bloc_pelouse.center_y = 32 # fixe ici
            # ajouter chaque bloc à la spriteList des tuiles (attribut de la fenêtre courante)
            self.tuiles.append(bloc_pelouse)
        ...
    
    def on_draw(self):
        ...
        self.tuiles.draw()
        ...
            


## Déplacement et interaction avec le décors - *3_main.py* - 

### Déplacement du personnage

On ajoute **Dans** la classe `MonJeu` les fonctions `on_key_press` et `on_key_release`.

Ces fonctions seront appelées *automatiquement* dès que l'utilisateur **enfonce** \[*press*\] ou **relâche** \[*release*\] une touche du clavier.

Par exemple, si l'utilisateur enfonce la flèche ↑ du clavier, alors la variable `key` prendra la valeur ... `arcade.key.UP`.

Si cela se produit, on veut déplacer notre personnage vers le haut: `self.sprite_joueur.center_y += <valeur>` **sauf si** ça lui fait traverser le décors!!!

### Interaction avec le décors

Comme il n'est pas du tout évident de gérer à la main le «décors» et le «**sauf si**» précédent, on utilise un gestionnaire spécialisé appelé `PhysicsEngineSimple` qui va s'occuper de ses choses pour nous.

Cela se passe en trois temps:

1. créer ce gestionnaire et le placer dans un attribut de la fenêtre courante: 

   `self.physics_engine = arcade.PhysicsEngineSimple(<sprite personnage>, <sprites du decors>)

2. créer une fonction `on_update` dans la classe `MonJeu`: elle sera appelée automatiquement à intervalle régulier

3. appeler, depuis la fonction précédente, la méthode `update` de `physics_engine`.

Voici les portions de code pertinentes:

```python
    def __init__(self):
        ...
        self.physics_engine = None
        ...
    def setup(self):
        ...
        self.physics_engine = arcade.PhysicsEngineSimple(
            self.sprite_joueur,  # personnage
            self.tuiles  # obstacles ou sols
        )
        ...
    def on_update(self, delta_time):
        ...
        self.physics_engine.update()
        ...↑
```

### Retour au déplacement du personnage

Grâce au «physics_engine», nous pouvons gérer le clavier sans nous préoccuper des obstacles.

Mais plutôt que de modifier le `.center_y` de notre sprite, nous modifions son `.change_y`. Cela donne:

```python
    def on_key_press(self, key, modifiers): # ne pas se préoccuper de modifiers...
        ...
        if key == arcade.key.UP:
            self.sprite_joueur.change_y = VITESSE_PERSONNAGE # constante à définir en début de fichier
        if key == arcade.key.DOWN:
            self.sprite_joueur.change_y = -VITESSE_PERSONNAGE
        ...
    def on_key_release(self, key, modifiers):
        if key == arcade.key.UP:
            self.sprite_joueur.change_y = 0    
        ...
```

## Ajuster la fenêtre de vue au déplacement du personnage - *4_main.py*

L'objectif est d'«agrandir» notre petit monde. Précisément, nous souhaiterions que la vue du jeu change lorsque le personnage approche des bords.

*Initialement*, la fenêtre de vue \[*viewport*\] est un rectangle défini par quatre nombres:
- bord gauche: xmin = 0
- bord bas: ymin = 0
- bord droit: xmax = `LARGEUR_ECRAN`
- bord haut: ymax = `HAUTEUR_ECRAN`


Mais arcade propose la méthode `arcade.set_viewport(xmin, ymin, xmax, ymax)` pour recadrer la fenêtre d'affichage.

Ainsi, **pour le côté gauche**, on commence par définir une marge minimum - `MARGE_GAUCHE_VUE` - entre le bord gauche de la vue et le personnage.

Puis, on ajoute l'attribut `self.xmin` à la fenêtre qu'on initialise à `0`.

Ensuite, dans `on_update`, on vérifie si le personnage (son bord gauche) entre dans cette marge (trop proche du bord gauche).

Si c'est le cas, on recalcule `self.xmin` et on règle la fenêtre de vue de façon que sa distance au bord gauche de la fenêtre soit précisément `MARGE_GAUCHE_VUE`. Cela donne quelquechose comme:

```python
    def on_update(self, time_delta):
        ...
        if self.sprite_joueur.left < self.xmin + MARGE_GAUCHE_VUE:
            # le personnage est trop proche du bord gauche!! On recadre
            self.xmin = self.sprite_joueur.left - MARGE_GAUCHE_VUE
            # et on règle la vue
            arcade.set_viewport(
                self.xmin,
                self.ymin,
                self.xmin + LARGEUR_ECRAN # xmax
                self.ymin + HAUTEUR_ECRAN # ymax
            )
```

Reste à faire de même pour les trois autres bords.

## Un personnage qui «saute» - *5_main.py*

Pour réaliser cela facilement, on va changer le moteur `PhysicsEngineSimple` par le moteur `PhysicsEnginePlatformer` lequel prend en compte la gravité. Reste alors à préciser le comportement lors de l'appui sur la touche ↑. 

On commence par définir de nouvelles constantes: `GRAVITE` et `VITESSE_SAUT_PERSONNAGE` puis:

```python
    ...
    def setup(self):
        ...
        self.physics_engine = arcade.PhysicsEnginePlatformer(
            self.sprite_joueur,  # personnage
            self.tuiles,  # obstacles ou sols
            GRAVITE  # force de la gravité
        )
    ...
    def on_key_press(self,...):
        ...
        if key == arcade.key.UP:
            # On vérifie que le joueur peut sauter
            if self.physics_engine.can_jump():
                self.sprite_joueur.change_y = VITESSE_SAUT_PERSONNAGE
        ...
```

## Du son et des pièces d'argent - *6_main.py*

Pour le son, on charge un fichier son avec `mon_saut = arcade.load_sound(chemin)`, puis on déclenche avec `arcade.play_sound(mon_son)` au moment propice (par exemple lors d'un saut).

Pour les pièces, on procède de manière similaire aux tuiles (herbe et obstacle) pour les placer, les dessiner... Remarquez qu'on les groupe dans une `SpriteList` dédiée.

Reste à gérer dans `on_update` la collision entre le personnage et les pièces avec `arcade.check_for_collision_with_list(un_sprite, des_sprites)` qui renvoie une liste de sprite en collision avec `un_sprite`. 

Dans les grandes lignes, cela donne:

```python
    def on_update(...):
        
        # on récupère
        pieces = arcade.check_for_collision_with_list(self.sprite_joueur, self.pieces)
        # et on agit
        for piece in pieces:
            piece.remove_from_sprite_lists()
            arcade.play_sound(self.son_collecte_piece)
```